In [2]:
import os
import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

In [3]:
def calculate_energy(subband):
    if len(subband.shape) == 1:
        R, C = 1, len(subband)
    else:
        R, C = subband.shape
    return np.sum(np.abs(subband)) / (R * C)

def dwt_feature_extraction(image, levels):
    imagecv = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    coeffs = pywt.wavedec2(imagecv, 'haar', level=levels)
    energy_vector = []

    for level in range(levels + 1):
        for subband in coeffs[level]:
            energy = calculate_energy(subband)
            energy_vector.append(energy)

    return energy_vector

In [4]:
def svd_feature_extraction(image):
    imagecv = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    U, S, VT = np.linalg.svd(imagecv)
    eigen_vector = VT.T[:, 0]

    return eigen_vector

In [5]:
def fuse_vectors(energy_vector, eigen_vector):
    return np.concatenate((energy_vector, eigen_vector))

In [23]:
def Learning_phase(fingerprint_samples):
    database = []
    for fingerprint_sample in fingerprint_samples:
        energy_vector = dwt_feature_extraction(fingerprint_sample[0], 6)
        eigen_vector = svd_feature_extraction(fingerprint_sample[0])

        feature_vector = fuse_vectors(energy_vector, eigen_vector)

        database.append({'feature_vector': feature_vector, 'class': fingerprint_sample[1]})
    return database


In [35]:
def classify_unknown_fingerprint(unknown_fingerprint, database, k_neighbors=3):
    energy_vector = dwt_feature_extraction(unknown_fingerprint, 6)
    eigen_vector = svd_feature_extraction(unknown_fingerprint)

    feature_vector = fuse_vectors(energy_vector, eigen_vector)
    feature_vector_length = len(database[0]['feature_vector'])
    
    X_train = np.array([entry['feature_vector'] for entry in database if len(entry['feature_vector']) == feature_vector_length])
    y_train = np.array([entry['class'] for entry in database if len(entry['feature_vector']) == feature_vector_length])

    knn_classifier = KNeighborsClassifier(n_neighbors=k_neighbors)
    knn_classifier.fit(X_train, y_train)

    predicted_class = knn_classifier.predict([feature_vector])

    return predicted_class[0]

In [36]:
def get_fingerprint_class(img_path, socofing):
    filename, _ = os.path.splitext(os.path.basename(img_path))

    if socofing:
        return 1
    else:
        subject_id, age, etc = filename.split('_')

        if int(age) <= 12 :
          ageClass = 0
        elif int(age) <= 19:
          ageClass = 1
        elif int(age) <= 25:
          ageClass = 2
        elif int(age) <= 35:
          ageClass = 3
        elif int(age) > 35:
          ageClass = 4
    
    return ageClass

In [37]:
def loading_data(path, socofing):
    data = []
    for img in os.listdir(path):
            ageclass = get_fingerprint_class(os.path.join(path, img), socofing)            
            data.append([os.path.join(path, img), ageclass])
    return data

In [39]:
#load images where age is know
pathToImage = "./Altered/Altered-Easy"

fingerprint_samples = loading_data(pathToImage, True)
print(fingerprint_samples[0])

print("Learning phase START")
database = Learning_phase(fingerprint_samples)
print(len(database))
print("Learning phase DONE")


#load unknown image to classify
unknown_fingerprint = "unknow_fingerprint.BMP"
predicted_class = classify_unknown_fingerprint(unknown_fingerprint, database)

print(f"The unknown fingerprint is classified as: {predicted_class}")

['./Altered/Altered-Easy\\100__M_Left_index_finger_CR.BMP', 1]
Learning phase START
17931
Learning phase DONE
The unknown fingerprint is classified as: 1
